# Comparison for LogSumExp

The LogSumExp-problem with linear consraints:
	$$\log(1 + \sum\limits_{k=1}^n \exp(\alpha x_k)) + \frac{\beta}{2} \|\textbf{x}\|_2^2 \rightarrow \min_{\textbf{x}\in \mathbb{R}^N}$$
	$$\text{s.t.} (\textbf{b}_j, \textbf{x}) - c_j <= 0, j = 1\dots M$$
	
This problem can be converted into saddle point problem:
	$$\min_{\textbf{x}} \max_{\textbf{y}\in\mathbb{R}^M_+} S(\textbf{x},\textbf{y}),$$
where $S(\textbf{x}, \textbf{y}) = r(\textbf{x})+F(\textbf{x}, \textbf{y}) - h(\textbf{y})$
	$$r(\textbf{x}) = \log(1 + \sum\limits_{k=1}^n \exp(\alpha x_k)) + \frac{\beta}{2} \|\textbf{x}\|_2^2$$
	$$F(\textbf{x}, \textbf{y}) = \sum\limits_{j=1}^M x_j(\textbf{b}_j, \textbf{y}) = (\textbf{x}, B\textbf{y})$$
	$$h(\textbf{y}) = (\textbf{c}, \textbf{y})$$

So N is dimensional of external variable, M - internal. But we will consider the external problem as help problem.

In [1]:
import numpy as np
from TestFunctions import LogSumExp_difalpha as LogSumExp
from TestFunctions.TestFunctions import TestFunction
from Solvers import GradientMethods
from Solvers.HalvCube import Dichotomy_exact
from Solvers.GradientMethods import FGM_internal
from Solvers.Ellipsoids import ellipsoid
import matplotlib.pyplot as plt
%matplotlib notebook
import comparison
import imp

from comparison import method_comparison, create_methods_dict

In [2]:
def get_LSE(N = 100, M = 2, alpha = 0.001, beta = 0.5e-3, k = 1, eps = 1e-3):
    """
    The function is to get class of LSE. All parameters mathes the
    parameters above.
    
    N is dimensional of primal problem
    M is number of constraints. It is also the dual dimension
    alpha is parameter of LogSumExp problem
    beta is coefficient of l2-regularization for primal problem
    
    k is parameter for to generate matrix B
    """
    np.random.seed(N*M)
    alpha_ = np.random.uniform(size = (N,), low = -alpha, high = alpha)
    alpha = np.max(np.abs(alpha_))
    B = []
    for i in range(M):
        b = np.random.uniform(low = -k, high = 0, size = (N,)) * np.sign(alpha_)
        B.append(b)
    B = np.array(B)
    #B = np.random.uniform(low = -k, high = 0, size = (M, N))
    c = np.ones((M,))
    x_ = np.zeros((N,))
    f_ = LogSumExp.r(alpha, beta).get_value(x_)
    gamma = c.max()
    y_max = f_/gamma
    Q = [[0, y_max] for i in range(M)]
    Q = np.array(Q)
    size_domain = np.linalg.norm(Q[:,0]-Q[:,1])
    solver = FGM_internal
    Q = Q.copy()
    beta_ = 2 * eps/ (size_domain**2)
    #size_domain_y = alpha * np.sqrt(N)/beta
    size_domain_y = np.sqrt(size_domain * np.linalg.norm(B, ord = 2)) / beta
    LSE_y_external = TestFunction(r = LogSumExp.h(c, beta = beta_, size_domain = size_domain, y0 = (Q[:,0] + Q[:, 1])/2),
                    F = LogSumExp.F_tilde(B, size_domain_x = size_domain, size_domain_y = size_domain_y),
                    h = LogSumExp.r(alpha_, beta, size_domain_y),
                    solver = solver, 
                    #get_start_point = lambda x: (1/beta * -x.dot(B), alpha * np.sqrt(N)/beta))
                    get_start_point = lambda x: (np.zeros(B.shape[1]), size_domain_y))
    solver = ellipsoid
    Q = Q.copy()
    LSE_ellipsoid = TestFunction(r = LogSumExp.r(alpha_, beta, size_domain_y),
                    F = LogSumExp.F(B, size_domain),
                    h = LogSumExp.h(c, beta = beta_, size_domain = size_domain, y0 = (Q[:,0] + Q[:, 1])/2),
                    solver = solver, 
                    get_start_point = lambda x: ((Q[:,0] + Q[:, 1])/2, size_domain/2))
    solver = lambda func, grad, L, mu, start_point, cond: Dichotomy_exact().Halving(func, 
                                                                                    grad, 
                                                                                    L, mu, 
                                                                                    eps, 
                                                                                    start_point,
                                                                                    cond)
    Q = Q.copy()
    LSE_dichotomy = TestFunction(r = LogSumExp.r(alpha_, beta, size_domain_y),
                F = LogSumExp.F(B, size_domain_x = size_domain_y, size_domain_y = size_domain),
                h = LogSumExp.h(c, beta = beta_, size_domain = size_domain, y0 = (Q[:,0] + Q[:, 1])/2),
                solver = solver, 
                get_start_point = lambda x: ((Q[:,0] + Q[:, 1])/2, size_domain/2))
    solver = FGM_internal
    Q = Q.copy()
    LSE_fgm = TestFunction(r = LogSumExp.r(alpha_, beta, size_domain_y),
                F = LogSumExp.F(B, size_domain_x = size_domain_y, size_domain_y = size_domain),
                h = LogSumExp.h(c, beta = beta_, size_domain = size_domain, y0 = (Q[:,0] + Q[:, 1])/2),
                solver = solver, 
                get_start_point = lambda x: ((Q[:,0] + Q[:, 1])/2, size_domain/2))
    R = alpha * np.sqrt(N)/beta
    return (LSE_y_external, Q), (LSE_ellipsoid, R), (LSE_dichotomy, R), (LSE_fgm, R), (alpha_, B, c, beta)

In [3]:
def start_comparison(N = 100, M = 2, alpha = 1e-3, beta = 1e-5, k = 1e3, eps = 1e-3, time_max = 0.2, with_inverse = False):
    LSE_res = get_LSE(N = N, M = M, alpha = alpha, beta = beta, k = k, eps = eps)
    (LSE_y_external, Q), (LSE_ellipsoid, R), (LSE_dichotomy, R), (LSE_fgm, R), stop_cond_args = LSE_res
    keys = {"FGM":"y, x - FGM+FGM",
        "Ellipsoids": "y, x - Ellipsoids+FGM",
        "Dichotomy": "y, x - Dichotomy+FGM"}
    #keys = {"FGM":"y, x - FGM+FGM",
    #    "Ellipsoids": "y, x - Ellipsoids+FGM"}
    history = {}
    f_dict = {}
    methods = {}
    methods = create_methods_dict(LSE_y_external, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = time_max, stop_cond_args = stop_cond_args, keys = keys)
    for key in methods:
        cond = lambda y, R: LSE_y_external.L_xx * R <= 1e-3
        f = lambda y: LSE_y_external.get_value(y, LSE_y_external.get_solution(y, cond)[0])
        f_dict[key] = f
    if with_inverse:
        Q = np.vstack([np.zeros((N, )) - R * np.ones((N,)), np.zeros((N, )) + R * np.ones((N,))]).T
        methods["x, y - FGM+FGM"] = create_methods_dict(LSE_fgm, np.mean(Q, axis = 1),
                                  np.linalg.norm(Q[:,0]-Q[:,1]),
                                  Q, eps, history, time_max = time_max, inverse = True,  stop_cond_args = stop_cond_args, keys = {"FGM":"x, y - FGM+FGM"})["x, y - FGM+FGM"]
        cond = lambda y, R: LSE_fgm.L_xx * R <= eps
        f = lambda y: LSE_fgm.get_value(y, LSE_fgm.get_solution(y, cond)[0])
        f_dict["x, y - FGM+FGM"] = f
        methods["x, y - FGM+Ellipsoids"] = create_methods_dict(LSE_ellipsoid, np.mean(Q, axis = 1),
                                  np.linalg.norm(Q[:,0]-Q[:,1]),
                                  Q, eps, history, time_max = time_max, inverse = True,  stop_cond_args = stop_cond_args, keys = {"FGM":"x, y - FGM+Ellipsoids"})["x, y - FGM+Ellipsoids"]
        cond = lambda y, R: LSE_ellipsoid.L_yy * R <= eps
        f = lambda y: LSE_ellipsoid.get_value(y, LSE_ellipsoid.get_solution(y, cond)[0])
        f_dict["x, y - FGM+Ellipsoids"] = f
        Q = np.vstack([np.zeros((N, )) - R * np.ones((N,)), np.zeros((N, )) + R * np.ones((N,))]).T
        methods["x, y - FGM+Dichotomy"] = create_methods_dict(LSE_dichotomy, np.mean(Q, axis = 1),
                                  np.linalg.norm(Q[:,0]-Q[:,1]),
                                  Q, eps, history, time_max = time_max, inverse = True, stop_cond_args = stop_cond_args, keys = {"FGM":"x, y - FGM+Dichotomy"})["x, y - FGM+Dichotomy"]
        cond = lambda y, R: LSE_dichotomy.L_xx * R <= eps
        f = lambda y: LSE_dichotomy.get_value(y, LSE_dichotomy.get_solution(y, cond)[0])
        f_dict["x, y - FGM+Dichotomy"] = f
    #comparison.method_comparison(methods)
    for key in methods:
        print(key)
        methods[key]()
    return history, stop_cond_args

In [4]:
def parse_results(history, alpha_, B, c, beta, keys = ["Ellipsoids", "Dichotomy", "FGM"]):
    s_key = ""
    s_feas = ""
    s_time = ""
    s_value = ""
    s_n = ""
    for key in keys:
        s_key += key + "\t$"
        if "y, x" in key:
            lambda_, x = history[key][-1][0]
        else:
            x, lambda_ = history[key][-1][0]
        s_feas += str((B @ x <= c).any()) + "\t\t$"
        s_time += "%.3f"%(history[key][-1][1] - history[key][0][1]) + "\t\t$"
        x_ = alpha_ * x
        M = np.max(x_)
        x_ = x_ - M
        s_value+= "%.3f"%(M + np.log(np.exp(-M) + np.exp(x_).sum()) + beta * np.linalg.norm(x)**2) + "\t\t$"
        s_n += "%d"%(len(history[key])) + "\t\t$"
    print(s_key)
    print(s_feas)
    print(s_time)
    print(s_value)
    print(s_n)

In [5]:
time_max = 60

## Inverse Order $ \max_x \min_y$

In [6]:
M = 2
N = 100

In [7]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max, with_inverse = True)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM


/home/elias/Desktop/PROJECTS/SPP/SPP/Solvers/GradientMethods.py:40: RuntimeWarning: divide by zero encountered in double_scalars
  est = 2 * min(4 * L * R/N**2, L * R * np.exp(-N/2 * np.sqrt(mu/L)))


y, x - Dichotomy+FGM
x, y - FGM+FGM
x, y - FGM+Ellipsoids
x, y - FGM+Dichotomy


In [8]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$False		$False		$False		$
0.017		$0.258		$0.141		$60.001		$60.008		$60.004		$
4.615		$4.616		$4.616		$4.615		$4.615		$4.615		$
4		$123		$56		$50677		$6618		$4690		$


In [9]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max, with_inverse = True)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM
x, y - FGM+FGM
x, y - FGM+Ellipsoids


/home/elias/Desktop/PROJECTS/SPP/SPP/Solvers/Ellipsoids.py:81: RuntimeWarning: divide by zero encountered in true_divide
  _df = _df / (np.sqrt(abs(_df@H@_df)))
/home/elias/Desktop/PROJECTS/SPP/SPP/Solvers/Ellipsoids.py:82: RuntimeWarning: invalid value encountered in matmul
  x = x - 1/(n+1) * H @ _df
/home/elias/Desktop/PROJECTS/SPP/SPP/Solvers/Ellipsoids.py:83: RuntimeWarning: invalid value encountered in matmul
  H = n**2/(n**2 - 1)*(H - (2 / (n + 1)) * (H @ np.outer(_df, _df) @ H))


x, y - FGM+Dichotomy


In [10]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$False		$True		$True		$
3.295		$0.428		$0.192		$60.001		$60.007		$60.008		$
4.615		$4.615		$4.615		$4.615		$4.615		$4.615		$
531		$139		$60		$44265		$6061		$5493		$


## M= 2

In [11]:
M = 2

### N = 100

In [12]:
N = 100

In [13]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [14]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
0.016		$0.270		$0.136		$
4.615		$4.616		$4.616		$
4		$123		$56		$


In [15]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [16]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
3.318		$0.442		$0.195		$
4.615		$4.615		$4.615		$
531		$139		$60		$


In [17]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [18]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
60.004		$0.794		$0.675		$
4.615		$4.615		$4.615		$
7895		$200		$74		$


### N = 1000

In [19]:
N = 1000

In [20]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [21]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
0.028		$0.519		$0.262		$
6.909		$6.909		$6.909		$
4		$131		$62		$


In [22]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [23]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
0.457		$0.841		$0.437		$
6.909		$6.909		$6.909		$
40		$147		$68		$


In [24]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [25]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
60.010		$1.462		$1.143		$
6.909		$6.909		$6.909		$
4198		$195		$84		$


### N = 10000

In [26]:
N = 10000

In [27]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [28]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
0.362		$6.976		$3.833		$
9.210		$9.211		$9.211		$
4		$131		$66		$


In [29]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [30]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$False		$True		$
0.568		$15.322		$5.840		$
9.210		$9.210		$9.210		$
4		$171		$74		$


In [31]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [32]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
60.031		$24.047		$15.160		$
9.210		$9.210		$9.210		$
258		$207		$87		$


## M= 3

In [33]:
M = 3

### N = 100

In [34]:
N = 100

In [35]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [36]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
0.049		$0.626		$0.919		$
4.615		$4.615		$4.616		$
4		$255		$83		$


In [37]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [38]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
2.413		$1.065		$60.008		$
4.615		$4.615		$4.615		$
385		$344		$89		$


In [39]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [40]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
60.004		$1.927		$60.027		$
4.615		$4.615		$4.615		$
7789		$470		$89		$


### N = 1000

In [41]:
N = 1000

In [42]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [43]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
0.029		$1.311		$1.589		$
6.909		$6.909		$6.909		$
4		$326		$92		$


In [44]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [45]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$False		$True		$
0.500		$2.006		$60.030		$
6.909		$6.909		$6.909		$
43		$347		$98		$


In [46]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [47]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
60.011		$3.633		$60.050		$
6.909		$6.909		$6.909		$
4139		$482		$98		$


### N = 10000

In [48]:
N = 10000

In [49]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [50]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
0.350		$20.656		$20.202		$
9.210		$9.211		$9.211		$
4		$342		$98		$


In [51]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [52]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$False		$True		$
0.611		$37.072		$60.325		$
9.210		$9.210		$9.210		$
4		$398		$110		$


In [53]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [54]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
60.222		$57.583		$60.274		$
9.210		$9.210		$9.210		$
260		$485		$110		$


## M= 4

In [55]:
M = 4

### N = 100

In [56]:
N = 100

In [57]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [58]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
0.051		$1.138		$7.074		$
4.615		$4.615		$4.616		$
4		$478		$110		$


In [59]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [60]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
3.160		$1.931		$60.022		$
4.615		$4.615		$4.615		$
495		$615		$114		$


In [61]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [62]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$False		$True		$
60.006		$3.494		$60.019		$
4.615		$4.615		$4.615		$
7791		$865		$114		$


### N = 1000

In [63]:
N = 1000

In [64]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [65]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
0.029		$2.237		$12.276		$
6.909		$6.909		$6.909		$
4		$541		$122		$


In [66]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [67]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$False		$True		$
0.520		$3.809		$60.031		$
6.909		$6.909		$6.909		$
44		$646		$130		$


In [68]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [69]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
60.007		$6.200		$60.048		$
6.909		$6.909		$6.909		$
4104		$815		$130		$


### N = 10000

In [70]:
N = 10000

In [71]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [72]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
0.357		$38.229		$60.149		$
9.210		$9.211		$27.879		$
4		$597		$104		$


In [73]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = time_max)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [74]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
0.589		$60.005		$60.154		$
9.210		$9.211		$25.489		$
4		$641		$105		$


In [77]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 200)
alpha_, B, c, beta = stop_cond_args

y, x - FGM+FGM
y, x - Ellipsoids+FGM
y, x - Dichotomy+FGM


In [78]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$
False		$True		$True		$
200.151		$102.533		$200.326		$
9.210		$9.210		$9.211		$
768		$833		$139		$
